In [3]:
%cd ../

/home/chris/fungiclef-2024


/home/chris/fungiclef-2024/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import pyspark.sql.functions as f
import pyspark.ml as ml
from pyspark.sql.types import ArrayType, FloatType, IntegerType
import fungiclef.transforms as trans
from fungiclef.utils import get_spark, read_config

/home/chris/fungiclef-2024/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Config for paths
config = read_config(path='fungiclef/config.json')

FileNotFoundError: [Errno 2] No such file or directory: 'fungiclef/config.json'

In [5]:
spark = get_spark(**{
    "spark.sql.parquet.enableVectorizedReader": False, 
})
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/07 21:04:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/07 21:04:22 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
24/04/07 21:04:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/07 21:04:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [6]:
gcs_training_path = config["gs_paths"]["train"]["raw_parquet"]


train_df = spark.read.parquet(gcs_training_path)
train_df.printSchema()
train_df.count()

root
 |-- image_path: string (nullable = true)
 |-- data: binary (nullable = true)
 |-- observationID: long (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- day: double (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- locality: string (nullable = true)
 |-- taxonID: double (nullable = true)
 |-- scientificName: string (nullable = true)
 |-- kingdom: string (nullable = true)
 |-- phylum: string (nullable = true)
 |-- class: string (nullable = true)
 |-- order: string (nullable = true)
 |-- family: string (nullable = true)
 |-- genus: string (nullable = true)
 |-- specificEpithet: string (nullable = true)
 |-- taxonRank: string (nullable = true)
 |-- species: string (nullable = true)
 |-- level0Gid: string (nullable = true)
 |-- level0Name: string (nullable = true)
 |-- level1Gid: string (nullable = true)
 |-- level1Name: string (nullable = true)
 |-- level2Gid: string (nullable = true)
 |-- level2Name: string (nullable = tr

295938

# Data Explore

In [ ]:
sum_poisonous = train_df.select(f.sum('poisonous')).collect()
percentage_poisonous = sum_poisonous[0][0] / train_df.count()
print(f"{percentage_poisonous} are poisonous")

0.043850401097527185 are poisonous


In [16]:
train_df.groupBy('family').count().orderBy(f.desc('count')).show()

+----------------+-----+
|          family|count|
+----------------+-----+
|     Russulaceae|19144|
|     Agaricaceae|15843|
|Tricholomataceae|15693|
|    Polyporaceae|13956|
|      Boletaceae|13589|
|  Hygrophoraceae|13347|
|      Mycenaceae|12169|
| Psathyrellaceae| 8657|
|Hymenogastraceae| 8621|
| Fomitopsidaceae| 8328|
|  Cortinariaceae| 8085|
|     Meruliaceae| 5562|
|Hymenochaetaceae| 5518|
| Physalacriaceae| 5451|
|     Amanitaceae| 5442|
|  Strophariaceae| 5232|
|    Parmeliaceae| 4874|
|  Entolomataceae| 4104|
|     Inocybaceae| 4054|
|      Pluteaceae| 4037|
+----------------+-----+
only showing top 20 rows



# Pick 

In [28]:
# Only take fungi of following three families
considered_families = ['Russulaceae', 'Boletaceae', 'Amanitaceae'
                       ]

family_subset = train_df.filter(train_df.family.isin(considered_families))                       

In [34]:
family_subset.count()

38175

In [36]:
# only take fungi of the following species
selected_mushrooms = ['Neoboletus luridiformis (Rostk.) Gelardi, Simonini & Vizzini, 2014',
                      'Imleria badia (Fr.) Vizzini, 2014',
                      'Amanita muscaria (L.) Lam., 1783',
                      'Russula ochroleuca (Pers.) Fr.',
                      'Russula nigricans (Bull.) Fr.',
                      'Lactarius blennius (Fr.) Fr.'
                      ]

species_subset = family_subset.filter(family_subset.scientificName.isin(selected_mushrooms))

In [38]:
# train test split with pyspark

# first, shuffle
species_subset_shuffled = species_subset.select("*").orderBy(f.rand())

# split by 80/20
splits = species_subset_shuffled.randomSplit(weights=[0.8,0.2], seed=42)

dev_train = splits[0]
dev_val = splits[1]

# Data Explore Dev Set

In [45]:
print(f"Length of whole dev_set: {species_subset.count()}")
print(f"Length of dev_train: {dev_train.count()}")
print(f"Length of dev_val: {dev_val.count()}")

Length of whole dev_set: 4643


Length of dev_train: 3723


Length of dev_val: 920


In [46]:
# posionus in dev train
sum_poisonous_dev = dev_train.select(f.sum('poisonous')).collect()[0][0]
percentage_poisonous_dev = sum_poisonous_dev/dev_train.count()
print("Poisonous in dev_train: ", sum_poisonous_dev)
print("Poisonous in dev_train in percent: ", percentage_poisonous_dev)
print("Poisonous in whole dataset in percent: ", percentage_poisonous)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Poisonous in dev_train:  786
Poisonous in dev_train in percent:  0.21112006446414183


NameError: name 'percentage_poisonous' is not defined

# Store dev set

In [43]:
dev_train.write.mode("overwrite").parquet(config['gs_paths']['parquet']['training_dev'])

In [44]:
dev_val.write.mode("overwrite").parquet(config['gs_paths']['parquet']['test_dev'])

In [49]:
train_df.limit(3).show(n=3)

+--------------------+--------------------+-------------+------+-----+----+-----------+---------------+-------+--------------------+-------+-------------+---------------+---------------+----------------+-----------+---------------+---------+--------------------+---------+----------+---------+-----------+----------+----------+-----------------+--------------------+-------------------+-----------------+---------+----------+--------------------+--------+-------------+---------+
|          image_path|                data|observationID|  year|month| day|countryCode|       locality|taxonID|      scientificName|kingdom|       phylum|          class|          order|          family|      genus|specificEpithet|taxonRank|             species|level0Gid|level0Name|level1Gid| level1Name| level2Gid|level2Name|    ImageUniqueID|           Substrate|       rightsHolder|         Latitude|Longitude|CoorUncert|             Habitat|class_id|MetaSubstrate|poisonous|
+--------------------+------------------